<a href="https://colab.research.google.com/github/cipriangerea/quant/blob/main/StrategyAllocationVis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import files

import ipywidgets as widgets
from IPython.display import display
import pandas as pd
import plotly.express as px
from io import StringIO
import tkinter as tk
from tkinter import filedialog
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import yfinance as yf
from datetime import datetime, timedelta
from io import StringIO





In [ ]:
# Upload files from your local system
uploaded = files.upload()

# Assuming a single file is uploaded, load it into a DataFrame
filename = next(iter(uploaded))

In [ ]:
#load into dataframe
if 'Portfolio Visualizer' in filename:
  # this is fron QuantMage
  df = pd.read_csv(filename, parse_dates=['Start Date'])
  rows = []

  for _, row in df.iterrows():
      date = row['Start Date']
      assets = row['Assets'].split(', ')
      weights = [float(weight.replace('%', '')) for weight in row['Weights'].split(', ')]
      for asset, weight in zip(assets, weights):
        rows.append({'Start Date': date, 'Asset': asset, 'Weight': weight})

  # Convert the list to DataFrame
  expanded_df = pd.DataFrame(rows)
  expanded_df.rename(columns={'Start Date': 'Date'}, inplace=True)
  df = expanded_df.pivot(index='Date', columns='Asset', values='Weight').fillna(0)

  # whatever doesn't add to 100% goes into $USD
  if 'BRK/B' in df.columns:
    df.rename(columns={'BRK/B': 'BRK-B'}, inplace=True)

  columns_to_process = [col for col in df.columns if col not in ['Date']]
  df['$USD'] = 0.0
  for _, row in df.iterrows():
    sum = 0.0
    for col in columns_to_process:
      sum += row[col]
    row['$USD'] = 100.0 - sum

else:
  df = pd.read_csv(filename, parse_dates=['Date'])
  df.drop('Day Traded', axis=1, inplace=True)
  df.set_index('Date', inplace=True)

  if 'BRK/B' in df.columns:
      df.rename(columns={'BRK/B': 'BRK-B'}, inplace=True)
  # clean the data
  # Exclude 'Date' and 'Day Traded' from the columns to be processed
  columns_to_process = [col for col in df.columns if col not in ['Date', 'Day Traded']]

  # Process the data: convert '-' to 0 and percentages to floats
  for col in columns_to_process:
      df[col] = df[col].replace('-', '0%').str.rstrip('%').astype('float')

  if '$USD' in df.columns:
        # If column exists, fill missing values with 0
        df['$USD'].fillna(0, inplace=True)
  else:
        # If column does not exist, create it and fill with 0s
        df['$USD'] = 0





In [ ]:
# let us use yfinance to compute profit stats
# Extract unique tickers

tickers = df.columns
unique_tickers = {ticker for ticker in tickers if ticker != '$USD'}  # Assume '$USD' is not a ticker

start_date = df.index.min()
end_date = df.index.max() + timedelta(days=1)  # Adding 3 days buffer

# Fetch historical prices
prices = yf.download(list(unique_tickers), start=start_date, end=end_date)['Adj Close']
prices['$USD'] = 1.0
prices=prices[tickers]

In [ ]:
df.sort_index(inplace=True)
prices.sort_index(inplace=True)

# Initialize the portfolio value DataFrame
portfolio_values = pd.Series(index=df.index, dtype=float)
portfolio_values.iloc[0] = 100000  # Starting with $100,000

# Initialize DataFrame for tracking individual stock values
stock_gains_total_df = pd.DataFrame(index=df.index, columns=prices.columns, dtype=float)
stock_gains_pct_df = pd.DataFrame(index=df.index, columns=prices.columns, dtype=float)


# Iterate over each trading day
for i in range(1, len(df)):
    # Previous day's total portfolio value
    previous_total_value = portfolio_values.iloc[i-1]

    # Today's allocations and prices
    allocations_today = df.iloc[i, :]/100
    prices_today = prices.loc[df.index[i]]

    # Yesterday's allocations & prices
    allocations_yday = df.iloc[i-1, :]/100
    prices_yday = prices.loc[df.index[i-1]]

    # Calculate the money allocated to each ticker at end of yesterday
    money_allocated_yday = previous_total_value * allocations_yday

    # Calculate shares held for each ticker today (until right before close)
    shares_today = money_allocated_yday.divide(prices_yday, fill_value=0)

    # Estimate the total value at the end of today
    portfolio_values.iloc[i] = (shares_today * prices_today).sum()

    # an estimate of the gains, assuming that yesterday's holding
    # are held at today's closing prices
    gains_today = (prices_today - prices_yday) * shares_today
    stock_gains_total_df.iloc[i] = gains_today

# Display the portfolio value
print(portfolio_values.tail())

In [ ]:
# Plotting the portfolio value

# Assuming 'portfolio_values' DataFrame has a column named 'Value' and the index is 'Date'
fig = px.line(portfolio_values, x=portfolio_values.index, y=portfolio_values, title='Dynamic Portfolio Value Over Time')

# Update layout and axes titles
fig.update_layout(
    xaxis_title='Date',
    yaxis_title='Portfolio Value ($)',
)

# Adding grid lines - Plotly has gridlines by default, but here's how to make sure:
fig.update_xaxes(showgrid=True, gridwidth=1, gridcolor='LightGrey')
fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor='LightGrey')

# Add title and format axes
fig.update_layout(
    hovermode="x unified",
    xaxis=dict(
        rangeslider=dict(
            visible=True
        ),
        type="date"
    ),
    # yaxis=dict(fixedrange=True)   # Prevents zooming on the y-axis
    yaxis=dict(),   # Prevents zooming on the y-axis
    yaxis_type="log",
    dragmode='pan'
)

# Show plot
fig.show()

In [ ]:
# Plot an area chart for the processed columns
fig = px.area(df, y=columns_to_process,
                title='Portfolio Allocation Over Time',
                labels={'value': 'Allocation', 'variable': 'Asset'})

# Customize hover information
fig.update_traces(
    hoverinfo="x+y+name",
    hovertemplate="%{y}<extra>%{data.name}</extra>"
)

# Add title and format axes
fig.update_layout(
    hovermode="x unified",
    xaxis=dict(
        rangeslider=dict(
            visible=True
        ),
        type="date"
    ),
    #xaxis=dict(fixedrange=False),  # Allows x-axis to be zoomable
    yaxis=dict(fixedrange=True)   # Prevents zooming on the y-axis
)

fig.show()

In [ ]:
filtered_df = stock_gains_total_df[stock_gains_total_df!=0]

cnt = filtered_df.count()
sum = filtered_df.sum()
cnt_pos = stock_gains_total_df[stock_gains_total_df>0].count()
cnt_neg = stock_gains_total_df[stock_gains_total_df<0].count()

stats_df = pd.DataFrame({
    'Count' : cnt,
    'CountPos' : cnt_pos,
    'CountNeg' : cnt_neg,
    'Profits' : sum,
    # 'AvgAlloc' : avgs,
}).sort_values(by=['Profits'])

stats_df['Profits'] = stats_df['Profits'].map(lambda x: f"${x:,.2f}")
# stats_df['AvgAlloc'] = stats_df['AvgAlloc'].map(lambda x: f"${x:,.2f}%")


print(stats_df)